In [7]:
import sys, os
import argparse

sys.path.append("../../NPTFit/")

import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import pandas as pd

from NPTFit import nptfit # module for performing scan
from NPTFit import create_mask as cm # module for creating the mask
from NPTFit import psf_correction as pc # module for determining the PSF correction
from NPTFit import dnds_analysis # module for analysing the output

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
def make_dirs(dirs):
    """ Creates directories if they do not already exist 
    """

    for d in dirs:
        if not os.path.exists(d):
            try:
                os.mkdir(d)
            except OSError as e:
                if e.errno != 17:
                    raise   

In [9]:
x_counts, y_counts, error_L, error_H, x_errors_L, x_errors_H = \
[np.array([  1.36887451e-10,   2.56502091e-10,   4.80638086e-10,
          9.00628020e-10,   1.68761248e-09,   3.16227766e-09,
          5.92553098e-09,   1.11033632e-08,   2.08056754e-08,
          3.89860370e-08,   7.30527154e-08]),
 np.array([  1.04000127e+08,   1.83397053e+08,   9.65856820e+07,
          1.51198295e+07,   4.76804443e+06,   9.78677656e+05,
          2.08916332e+05,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00]),
 np.array([  2.14237668e+07,   2.08831658e+07,   1.10708578e+07,
          3.18362798e+06,   1.29929969e+06,   4.21069315e+05,
          1.34538182e+05,  -5.57461814e-04,  -2.97500603e-04,
         -1.58767124e-04,  -8.47292389e-05]),
 np.array([  2.63822671e+07,   2.34164673e+07,   1.24232945e+07,
          3.93887993e+06,   1.71404939e+06,   6.58746511e+05,
          2.74201404e+05,   1.02159419e+05,   5.45194091e+04,
          2.90953689e+04,   1.55273233e+04]),
 np.array([  3.68874510e-11,   6.91203483e-11,   1.29518913e-10,
          2.42694796e-10,   4.54765736e-10,   8.52147960e-10,
          1.59676969e-09,   2.99205487e-09,   5.60656455e-09,
          1.05056783e-08,   1.96857231e-08]),
 np.array([  5.04942913e-11,   9.46170829e-11,   1.77295138e-10,
          3.32218719e-10,   6.22517224e-10,   1.16648362e-09,
          2.18577733e-09,   4.09574765e-09,   7.67468330e-09,
          1.43809553e-08,   2.69472846e-08])]

In [26]:
xsecs_ary = [0] + list(np.logspace(-27,-24.5,10))
xsec_print = np.round(np.log10(xsecs_ary),1)

fix_1ph = 0
i_ebin = 0

psf_king_list = []
i_xsec_list = []
ps_mask_list = []
nexp_list = []
plots_list = []

github_link = 'https://github.com/laurajchang/NPTF-IG-Check/tree/master/runs_data/'

for nexp in [1]:
    for ps_mask in ["3fgl_0p8deg"]:  
        for psf_king in [1]:
            for i_xsec in range(6):
                for i_mc in [0]:

                    xsec_inj = xsecs_ary[i_xsec]

                    psf_king_list.append(psf_king)
                    i_xsec_list.append(xsec_inj)
                    ps_mask_list.append(ps_mask)
                    nexp_list.append(nexp)

                    tag = 'mc_' + str(i_mc) + "_xsec_" + str(i_xsec) + "_king_" + str(psf_king) + "_mask_" + ps_mask
                    dir_tag = 'plots/' + tag +  '_nexp_' + str(nexp) + '/'

                    make_dirs([dir_tag])
                    n = nptfit.NPTF(work_dir='/scratch/sm8383/old_runs/NEXP1_new/', tag=tag) 
                    fermi_data = np.load('../data/MC_inj_data.npy')[i_mc, i_xsec, i_ebin]

                    fermi_exposure = np.load('../data/fermi_data/fermidata_exposure.npy')
                    n.load_data(fermi_data, fermi_exposure)

                    if ps_mask == "3fgl_0p95psf":
                        pscmask = np.array(np.load('../data/fermi_data/fermidata_pscmask.npy'), dtype=bool)
                    elif ps_mask == "3fgl_0p8deg":
                        pscmask = np.array(np.load('../data/mask_3fgl_0p8deg.npy'), dtype=bool)
                    elif ps_mask == "4fgl_0p8deg":
                        pscmask = np.array(np.load('../data/mask_4fgl_0p8deg.npy'), dtype=bool)

                    analysis_mask = cm.make_mask_total(band_mask = True, band_mask_range = 2,
                                                       mask_ring = True, inner = 0, outer = 30,
                                                       custom_mask = pscmask)
                    plots_mask = cm.make_mask_total(band_mask = True, band_mask_range = 2,
                                                       mask_ring = True, inner = 0, outer = 30)

                    n.load_mask(analysis_mask)

                    dif = np.load('../data/fermi_data/template_dif.npy')
                    iso = np.load('../data/fermi_data/template_iso.npy')
                    bub = np.load('../data/fermi_data/template_bub.npy')
                    gce = np.load('../data/SGH_Jfactor_map_NFW_gamma_1.2_baseline.npy')
                    dsk = np.load('../data/fermi_data/template_dsk.npy')
                    psc = np.load('../data/fermi_data/template_psc.npy')

                    n.add_template(dif, 'dif')
                    n.add_template(iso, 'iso')
                    n.add_template(bub, 'bub')
                    n.add_template(gce, 'gce')
                    n.add_template(dsk, 'dsk')
                    n.add_template(psc, 'psc')

                    # Remove the exposure correction for PS templates
                    rescale = fermi_exposure/np.mean(fermi_exposure)
                    n.add_template(gce/rescale, 'gce_np', units='PS')
                    n.add_template(dsk/rescale, 'dsk_np', units='PS')
                    n.add_template(iso/rescale, 'iso_np', units='PS')

                    n.add_poiss_model('dif', '$A_\mathrm{dif}$', [8,17], False)
                    n.add_poiss_model('iso', '$A_\mathrm{iso}$', [0,2], False)
                    n.add_poiss_model('gce', '$A_\mathrm{gce}$', [-4,2], True)
                    n.add_poiss_model('bub', '$A_\mathrm{bub}$', [0,2], False)
                    n.add_poiss_model('psc', '$A_\mathrm{psc}$', [0,2], False)

                    if fix_1ph:

                        n.add_non_poiss_model('gce_np',
                                            ['$A_\mathrm{gce}^\mathrm{ps}$','$n_1^\mathrm{gce}$','$n_2^\mathrm{gce}$','$n_3^\mathrm{gce}$','$S_b^{(1), \mathrm{gce}}$','$S_b^{(2), \mathrm{gce}}$'],
                                            [[-6,2],[2.05,15],[1.05,1.95],[np.log10(2.),np.log10(100.)]],
                                            [True,False,False,False,True,True],
                                            fixed_params = [[3,-10],[5,1.]])

                    else:

                        n.add_non_poiss_model('gce_np',
                                            ['$A_\mathrm{gce}^\mathrm{ps}$','$n_1^\mathrm{gce}$','$n_2^\mathrm{gce}$','$n_3^\mathrm{gce}$','$S_b^{(1), \mathrm{gce}}$','$S_b^{(2), \mathrm{gce}}$'],
                                            [[-6,2],[2.05,15],[1.05,1.95],[-1.95,0.95],[np.log10(2.),np.log10(100.)],[np.log10(0.1),np.log10(2.)]],
                                            [True,False,False,False,True,True])

                    n.add_non_poiss_model('dsk_np',
                                          ['$A_\mathrm{dsk}^\mathrm{ps}$','$n_1^\mathrm{dsk}$','$n_2^\mathrm{dsk}$','$n_3^\mathrm{dsk}$','$S_b^{(1), \mathrm{dsk}}$','$S_b^{(2), \mathrm{dsk}}$'],
                                          [[-6,2],[2.05,15],[1.05,1.95],[-1.95,0.95],[np.log10(2.),np.log10(100.)],[np.log10(0.1),np.log10(2.)]],
                                          [True,False,False,False,True,True])

                    n.add_non_poiss_model('iso_np',
                                          ['$A_\mathrm{iso}^\mathrm{ps}$','$n_1^\mathrm{iso}$','$n_2^\mathrm{iso}$','$n_3^\mathrm{iso}$','$S_b^{(1), \mathrm{iso}}$','$S_b^{(2), \mathrm{iso}}$'],
                                          [[-6,2],[2.05,15],[1.05,1.95],[-1.95,0.95],[np.log10(2.),np.log10(100.)],[np.log10(0.1),np.log10(2.)]],
                                          [True,False,False,False,True,True])

                    if psf_king:

                        # Define parameters that specify the Fermi-LAT PSF at 2 GeV
                        fcore = 0.748988248179
                        score = 0.428653790656
                        gcore = 7.82363229341
                        stail = 0.715962650769
                        gtail = 3.61883748683
                        spe = 0.00456544262478

                        # Define the full PSF in terms of two King functions
                        def king_fn(x, sigma, gamma):
                          return 1./(2.*np.pi*sigma**2.)*(1.-1./gamma)*(1.+(x**2./(2.*gamma*sigma**2.)))**(-gamma)

                        def Fermi_PSF(r):
                          return fcore*king_fn(r/spe,score,gcore) + (1-fcore)*king_fn(r/spe,stail,gtail)

                        # Modify the relevant parameters in pc_inst and then make or load the PSF
                        pc_inst = pc.PSFCorrection(delay_compute=True)
                        pc_inst.psf_r_func = lambda r: Fermi_PSF(r)
                        pc_inst.sample_psf_max = 10.*spe*(score+stail)/2.
                        pc_inst.psf_samples = 10000
                        pc_inst.psf_tag = 'Fermi_PSF_2GeV'
                        pc_inst.make_or_load_psf_corr()

                        # Extract f_ary and df_rho_div_f_ary as usual
                        f_ary = pc_inst.f_ary
                        df_rho_div_f_ary = pc_inst.df_rho_div_f_ary

                    else:

                        pc_inst = pc.PSFCorrection(psf_sigma_deg=0.1812)
                        f_ary, df_rho_div_f_ary = pc_inst.f_ary, pc_inst.df_rho_div_f_ary

                    n.configure_for_scan(f_ary, df_rho_div_f_ary, nexp=1)
#                         n.perform_scan()

                    try:

                        n.load_scan()

                        an = dnds_analysis.Analysis(n, mask=plots_mask)

#                         an.make_triangle()

#                         plt.savefig(dir_tag + "/corner.pdf")
#                         plt.close()

                        plt.figure(figsize=[8,6])

                        an.plot_source_count_median('dsk_np',smin=0.01,smax=1000,nsteps=1000,color='royalblue',spow=2,label='Disk PS')
                        an.plot_source_count_band('dsk_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.16,0.5,0.84],color='royalblue',alpha=0.15,spow=2)
                        an.plot_source_count_band('dsk_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.025,0.5,0.975],color='royalblue',alpha=0.1,spow=2)

                        an.plot_source_count_median('gce_np',smin=0.01,smax=1000,nsteps=1000,color='firebrick',spow=2,label='GCE PS')
                        an.plot_source_count_band('gce_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.16,0.5,0.84],color='firebrick',alpha=0.15,spow=2)
                        an.plot_source_count_band('gce_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.025,0.5,0.975],color='firebrick',alpha=0.1,spow=2)

                        an.plot_source_count_median('iso_np',smin=0.01,smax=1000,nsteps=1000,color='forestgreen',spow=2,label='Iso PS')
                        an.plot_source_count_band('iso_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.16,0.5,0.84],color='forestgreen',alpha=0.15,spow=2)
                        an.plot_source_count_band('iso_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.025,0.5,0.975],color='forestgreen',alpha=0.1,spow=2)

                        plt.errorbar(x_counts,x_counts**2*y_counts,xerr=[x_errors_L,x_errors_H],yerr=x_counts**2*np.array([error_L,error_H]), fmt='o', color='black', label='3FGL PS')

                        plt.yscale('log')
                        plt.xscale('log')
                        plt.xlim([1e-12,1e-7])
                        plt.ylim([1e-20,1e-10])

                        plt.axvline(1/np.mean(fermi_exposure), ls='--', color='grey', label='1-ph')

                        plt.tick_params(axis='x', length=5, width=2, labelsize=18)
                        plt.tick_params(axis='y', length=5, width=2, labelsize=18)
                        plt.ylabel('$F^2 dN/dF$ [counts cm$^{-2}$s$^{-1}$deg$^{-2}$]', fontsize=18)
                        plt.xlabel('$F$  [ph cm$^{-2}$ s$^{-1}$]', fontsize=18)
                        plt.legend(fancybox=True, loc='lower right');
                        plt.tight_layout()
                        plt.savefig(dir_tag + "/s2dnds.pdf")

                        plt.close()

#                         ###

#                         plt.figure(figsize=[8,6])

#                         an.plot_source_count_median('dsk_np',smin=0.01,smax=1000,nsteps=1000,color='royalblue',spow=0,label='Disk PS')
#                         an.plot_source_count_band('dsk_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.16,0.5,0.84],color='royalblue',alpha=0.15,spow=0)
#                         an.plot_source_count_band('dsk_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.025,0.5,0.975],color='royalblue',alpha=0.1,spow=0)

#                         an.plot_source_count_median('gce_np',smin=0.01,smax=1000,nsteps=1000,color='firebrick',spow=0,label='GCE PS')
#                         an.plot_source_count_band('gce_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.16,0.5,0.84],color='firebrick',alpha=0.15,spow=0)
#                         an.plot_source_count_band('gce_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.025,0.5,0.975],color='firebrick',alpha=0.1,spow=0)

#                         an.plot_source_count_median('iso_np',smin=0.01,smax=1000,nsteps=1000,color='forestgreen',spow=0,label='Iso PS')
#                         an.plot_source_count_band('iso_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.16,0.5,0.84],color='forestgreen',alpha=0.15,spow=0)
#                         an.plot_source_count_band('iso_np',smin=0.01,smax=1000,nsteps=1000,qs=[0.025,0.5,0.975],color='forestgreen',alpha=0.1,spow=0)

#                         plt.errorbar(x_counts,x_counts**0*y_counts,xerr=[x_errors_L,x_errors_H],yerr=x_counts**0*np.array([error_L,error_H]), fmt='o', color='black', label='3FGL PS')

#                         plt.yscale('log')
#                         plt.xscale('log')
#                         plt.xlim([1e-12,1e-7])
#                         plt.ylim([1e5,5e10])

#                         plt.axvline(1/np.mean(fermi_exposure), ls='--', color='grey', label='1-ph')

#                         plt.tick_params(axis='x', length=5, width=2, labelsize=18)
#                         plt.tick_params(axis='y', length=5, width=2, labelsize=18)
#                         plt.ylabel('$dN/dF$', fontsize=18)
#                         plt.xlabel('$F$  [ph cm$^{-2}$ s$^{-1}$]', fontsize=18)
#                         plt.legend(fancybox=True, loc='lower right');
#                         plt.tight_layout()
#                         plt.savefig(dir_tag + "/dnds.pdf")

#                         plt.close()

#                         ##

#                         intens_inj = np.median(an.return_intensity_arrays_poiss('gce'))/an.nptf.norms_poiss['gce']*xsec_inj/(2e-26)

#                         plt.hist(an.return_intensity_arrays_non_poiss('gce_np'), histtype='step', color='firebrick', label='GCE PS')
#                         plt.hist(an.return_intensity_arrays_poiss('gce'), histtype='step', color='grey', label='GCE DM')
#                         plt.axvline(intens_inj, ls='--', color='orange', label="Injected DM")

#                         plt.xlabel("Intensity [ph\,cm$^{-2}$\,s$^{-1}$]")
#                         # plt.xlim(0,4e-7)

#                         plt.legend(frameon=False)

#                         plt.tight_layout()
#                         plt.savefig(dir_tag + "/intens.pdf")

#                         plt.close()      


                        plots_list.append("[[dNdS]]({github}{dir}/dnds.pdf) [[S2dNdS]]({github}{dir}/s2dnds.pdf) [[Intens]]({github}{dir}/intens.pdf) [[Corner]]({github}{dir}/corner.pdf)".format(github=github_link,dir=dir_tag))
                    except:
                        plots_list.append("Failed")

/scratch/sm8383/anaconda3/envs/venv_smsharma/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log10
  


Loading the psf correction from: /home/sm8383/NPTF-Check/runs_data/psf_dir/Fermi_PSF_2GeV.npy
The number of parameters to be fit is 23
  analysing data from /scratch/sm8383/old_runs/NEXP1_new/chains/mc_0_xsec_0_king_1_mask_3fgl_0p8deg/.txt
Loading the psf correction from: /home/sm8383/NPTF-Check/runs_data/psf_dir/Fermi_PSF_2GeV.npy
The number of parameters to be fit is 23
  analysing data from /scratch/sm8383/old_runs/NEXP1_new/chains/mc_0_xsec_1_king_1_mask_3fgl_0p8deg/.txt
Loading the psf correction from: /home/sm8383/NPTF-Check/runs_data/psf_dir/Fermi_PSF_2GeV.npy
The number of parameters to be fit is 23
  analysing data from /scratch/sm8383/old_runs/NEXP1_new/chains/mc_0_xsec_2_king_1_mask_3fgl_0p8deg/.txt
Loading the psf correction from: /home/sm8383/NPTF-Check/runs_data/psf_dir/Fermi_PSF_2GeV.npy
The number of parameters to be fit is 23
  analysing data from /scratch/sm8383/old_runs/NEXP1_new/chains/mc_0_xsec_3_king_1_mask_3fgl_0p8deg/.txt
Loading the psf correction from: /home/s

In [23]:
xsec_list = [np.format_float_scientific(i_xsec_list[i], precision=2) for i in range(len(i_xsec_list))]

In [24]:
run_no = np.arange(len(nexp_list))
d = {'run':run_no, 'nexp':nexp_list, 'mask': ps_mask_list, 'King PSF': psf_king_list, 'xsec': xsec_list, 'plots':plots_list}
df = pd.DataFrame(data=d)

In [25]:
# Get column names
cols = df.columns

# Create a new DataFrame with just the markdown
# strings
df2 = pd.DataFrame([['---',]*len(cols)], columns=cols)

#Create a new concatenated DataFrame
df3 = pd.concat([df2, df])

#Save as markdown
df3.to_csv("README.md", sep="|", index=False)
